In [1]:
import os
import pandas as pd
import fetchData
import numpy as np
import glob
import Timestamp
from datetime import datetime, timezone, timedelta

In [2]:
data_type = 'TAF'
dataset_purposes = ['train', 'test']
TAF_airports = ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']
date_ranges = [[datetime(2022,9,1), datetime(2023,8,31)],[datetime(2022,9,25), datetime(2023,9,18)]]

In [3]:
def find_forecast_files(directory, date):
    # taf.20220901.00Z.txt
    pattern = os.path.join(directory, f"taf.{date.strftime("%Y%m%d")}.??Z.txt")
    return glob.glob(pattern)

In [4]:
def mean_excluding_special(series):
    # Filter out special values -1 and -2
    filtered = series[~series.isin([-1, -2])]
    
    # Check if the filtered series is empty or has only NaN values
    if filtered.empty or filtered.isna().all():
        return np.nan  # Return NaN if no valid values are present
    
    # Calculate the mean and return it as an integer if not NaN
    mean_value = filtered.mean()
    return int(mean_value) if not pd.isna(mean_value) else np.nan


In [5]:
def preprocess_tar_data(df, complete_df):
    # Rename and merge
    df.rename(columns={'date_time': 'timestamp_15mins'}, inplace=True)
    df = complete_df.merge(df, on='timestamp_15mins', how='left')

    # Expand cloud_layers column into sky_cover, altitude_ft, and cumulonimbus
    df['sky_cover'] = df['cloud_layers'].apply(lambda x: x[0]['sky_cover'] if isinstance(x, list) and x else np.nan).astype(float)
    df['altitude_ft'] = df['cloud_layers'].apply(lambda x: x[0]['altitude_ft'] if isinstance(x, list) and x else np.nan).astype(float)
    df['cumulonimbus'] = df['cloud_layers'].apply(lambda x: x[0]['cumulonimbus'] if isinstance(x, list) and x else np.nan).astype(float)

    # Calculate temperature delta
    df['temperature_delta'] = (df['max_temperature_time'] - df['min_temperature_time']).dt.total_seconds() / 3600  # In hours
     
    # Aggregate data every 15 minutes
    preprocessed_df = df.resample('15min', on='timestamp_15mins').agg({
        'visibility_meters': 'mean',
        'has_wind': 'max',  # Aggregates as 1 if any record within the interval has wind data
        'wind_direction': mean_excluding_special,  # Custom mean excluding special values
        'wind_is_variable': 'max',  # Aggregates as 1 if any record within the interval has variable wind
        'wind_speed_kt': 'mean',
        'wind_gust_kt': 'mean',
        'qnh_hpa': 'mean',
        'sky_cover': 'first',
        'altitude_ft': 'first',
        'cumulonimbus': 'first',
        'probability': 'first',
        'has_prob': 'first',
        'temperature_delta': 'first',
        'weather_score': 'mean'
    }).reset_index()
    return preprocessed_df


In [6]:
def get_final_df(df_list):
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # Set only `timestamp_15mins` as the index for resampling
    combined_df.set_index('timestamp_15mins', inplace=True)
    combined_df = combined_df.groupby("timestamp_15mins").agg({
        'visibility_meters': 'mean',
        'has_wind': 'max',  # Aggregates as 1 if any record within the interval has wind data
        'wind_direction': mean_excluding_special,  # Custom mean excluding special values
        'wind_is_variable': 'max',  # Aggregates as 1 if any record within the interval has variable wind
        'wind_speed_kt': 'mean',
        'wind_gust_kt': 'mean',
        'qnh_hpa': 'mean',
        'sky_cover': 'first',
        'altitude_ft': 'first',
        'cumulonimbus': 'first',
        'probability': 'first',
        'has_prob': 'first',
        'temperature_delta': 'first',
        'weather_score': 'mean'
    }).reset_index()

    columns_to_fill = ['visibility_meters', 'wind_speed_kt', 'qnh_hpa', 'wind_gust_kt',
                       'sky_cover', 'altitude_ft', 'cumulonimbus', 'temperature_delta', 'weather_score']  # Specify the columns you want to fill
    combined_df[columns_to_fill] = combined_df[columns_to_fill].ffill().bfill().infer_objects(copy=False)

    # Selectively interpolate for specific columns
    columns_to_interpolate = ['wind_direction', 'weather_score']  # Specify columns for interpolation
    combined_df[columns_to_interpolate] = combined_df[columns_to_interpolate].interpolate(method='linear').infer_objects(copy=False)
    
    # Handle missing or variable wind direction
    combined_df['wind_direction'] = combined_df['wind_direction'].apply(lambda x: -1 if x == 'VRB' or pd.isnull(x) else x)
    combined_df['has_wind'] = 1 if combined_df['wind_speed_kt'] is not None else 0
    combined_df['wind_is_variable'] = combined_df['wind_direction'].apply(lambda x: 1 if x == -1 else 0)

     # Set default probability and add 'has_prob' indicator
    combined_df['has_wind'] = combined_df['wind_speed_kt'].notna().astype(int)
    combined_df['probability'] = combined_df['probability'].fillna(0.9)
    combined_df['has_prob']  = combined_df['has_prob'].ffill().bfill().astype(bool)
    
    #combined_df = combined_df.ffill().infer_objects(copy=False)
    #combined_df = combined_df.bfill().infer_objects(copy=False)
    #combined_df = combined_df.interpolate(method='linear').infer_objects(copy=False)

    return combined_df

In [7]:
for dataset_purpose in dataset_purposes:
    date_range = date_ranges[0] if dataset_purpose == 'train' else date_ranges[1]
    current_date = date_range[0]
    
    while current_date <= date_range[1]:
        # Prepare the daily file path
        data_range = current_date.strftime("%Y-%m-%d")
        file_path = os.path.join('./', 'data', data_type, dataset_purpose, f'{data_range}.csv')
        
        if os.path.exists(file_path):
            print(file_path, "saved!")
            current_date += timedelta(days=1)
            continue  # Skip processing if file already exists

        # Set the directory for the dataset and look for forecast files
        directory = os.path.join(fetchData.get_defult_base_dir(), data_type, dataset_purpose)
        forecast_files = find_forecast_files(directory, current_date)
        
        if not forecast_files:
            current_date += timedelta(days=1)
            continue  # Skip the date if no forecast files are found
        df_list = []
        # Define the timestamp range for the day in 15-minute intervals
        start_date = pd.to_datetime(data_range).tz_localize('UTC')
        end_date = start_date + pd.Timedelta(days=1)
        complete_times = pd.date_range(start=start_date, end=end_date, freq='15min')[:-1]

        # Process each forecast file
        for forecast_file in forecast_files:
            file_name = os.path.splitext(os.path.basename(forecast_file))[0]  # Strip extension from filename
            df = fetchData.load_data(
                data_type=data_type,
                dataset_purpose=dataset_purpose,
                file_name=file_name
            )
            complete_index = pd.MultiIndex.from_product([complete_times], names=['timestamp_15mins'])
            complete_df = pd.DataFrame(index=complete_index).reset_index()
            preproceed_df = preprocess_tar_data(df=df, complete_df=complete_df)
            df_list.append(preproceed_df)
        
        final_df = get_final_df(df_list=df_list)

        file_path = os.path.join('./', 'data', data_type, dataset_purpose, f'{data_range}.csv')
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        final_df.to_csv(file_path, index=False)
        print(file_path, "saved!")
        # Increment the current date
        current_date += timedelta(days=1)

/tmp/ipykernel_54186/3038440032.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['has_prob']  = combined_df['has_prob'].ffill().bfill().astype(bool)


./data/TAF/train/2022-09-01.csv saved!


/tmp/ipykernel_54186/3038440032.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['has_prob']  = combined_df['has_prob'].ffill().bfill().astype(bool)


./data/TAF/train/2022-09-02.csv saved!


/tmp/ipykernel_54186/3038440032.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['has_prob']  = combined_df['has_prob'].ffill().bfill().astype(bool)


...

/tmp/ipykernel_54186/3038440032.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['has_prob']  = combined_df['has_prob'].ffill().bfill().astype(bool)


./data/TAF/test/2023-08-16.csv saved!


/tmp/ipykernel_54186/3038440032.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['has_prob']  = combined_df['has_prob'].ffill().bfill().astype(bool)


./data/TAF/test/2023-08-17.csv saved!
./data/TAF/test/2023-08-18.csv saved!


/tmp/ipykernel_54186/3038440032.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['has_prob']  = combined_df['has_prob'].ffill().bfill().astype(bool)
